# Camera calibration

In this notebook, the calibration images for the polychromatic beam set are processed for initial calibration of each of the three cameras.

---

## Environment setup

In [1]:
import ipywidgets as widgets
import numpy as np

from ipywidgets import (
                        fixed,
                        interact
                        )

from calib_utilities import (                    
                             processCamera,
                             showPoints,
                             calibCamera,
                             showCalibration,
                             showReprojection
                             )


# Data directories
inp_fld = 'inputs'
out_fld = 'outputs'

### Version information

In [2]:
%reload_ext watermark
%watermark -p ipywidgets,numpy,cv2,h5py,matplotlib,skimage

ipywidgets: 7.6.5
numpy     : 1.17.0
cv2       : 4.5.1
h5py      : 3.6.0
matplotlib: 3.4.3
skimage   : 0.18.1



## Calibration setup

The camera calibrations are done by locating dots on a known dot target. The camera calibration routine here is Zhang's method [[1]](#References), where the calibration plate (a 1 mm thick aluminum plate with a 19$\times$19 grid of 0.75 mm diameter holes that are 3.7 mm spaced apart) is rotated to 3 different orientations at the same fixed location, with each "view" captured by all the cameras simultaneously as an X-ray radiograph. As all views are collected simultaneously, the extrinsics calculated from each rotation is consistent for all cameras$-$therefore, the world origin is selectively chosen to be the view that gives the lowest pixel projection errors.

## Find the dots!

Let's first start the calibration process by finding the dots in each of the views for each camera. The calibration radiographs are located in the HDF5 files under the `/inputs` folder. For easier processing, the calibration is done on "masked" versions of the calibration plate views$-$each view of the calibration plate was manually masked beforehand such that all the views looked at the same dots for each camera separately. A dot below two fiducial marks on the plate ("lines" cut into the top edge) was taken to be the world origin and was kept consistent between all calibration plate views and cameras.

The dot finding routine is done in the `processCamera()` function (see: [calib_utilities.py](./calib_utilities.py)), which loads in each camera's images and takes as arguments the number of total dots for each view, the location of the origin dot in the grid, whether or not to flip the grid (for camera 2 looking at the back side), and whether or not to mask the image (necessary for some of the poorer quality images of cameras 1 and 3). The outputs of the function are the located dots in image coordinates (`imgPts`) and their corresponding location in world coordinates (`objPts`).

In [3]:
# Get the image and object points for each camera
# The passed in arguments (see processCamera()) were pre-determined--if 
# different images are used, you'll have to play around with the values!
imgPts_1, objPts_1 = processCamera('Cropped/Camera1', (11, 4), 6, 1, (0, 0, 1))
imgPts_2, objPts_2 = processCamera('Cropped/Camera2', (7, 5), 3, -1, (0, 0, 0))
imgPts_3, objPts_3 = processCamera('Cropped/Camera3', (10, 5), 5, 1, (0, 0, 1))

# Combine all found points
imgPts = [imgPts_1, imgPts_2, imgPts_3]
objPts = [objPts_1, objPts_2, objPts_3]

## Visualization

The dot finding routine in `processCamera()` uses a "blob detector" (see: `makeBlob()` in [calib_utilities.py](./calib_utilities.py)) which attempts to locate circular features in the images. As a sanity check, let's visually verify how well the located dots match against the radiographs.

In [4]:
# Visualizing the results through an interactive plot
# Make sure all points are found!
interact(
         showPoints, 
         cam=widgets.Dropdown(options=[1, 2, 3], description='Camera'), 
         view=widgets.IntSlider(min=1, max=3, step=1, description='View'),
         imgPts=fixed(imgPts),
         objPts=fixed(objPts)
         );

interactive(children=(Dropdown(description='Camera', options=(1, 2, 3), value=1), IntSlider(value=1, descripti…

## Calibration

Now that the dots are located, let's calibrate each of the cameras. In addition to the radial and tangential distortion models used in Zhang's seminal report, additional distortion models (rational, thin prism, and tilted) are also utilized to better map the non-linear projection between the scintillator plate (the focal/detection plane) and the camera image. The processing is done in `calibCamera()` (see: [calib_utilities.py](./calib_utilities.py)), which takes as inputs the image and object points for each camera. The outputs are tuples containing the camera calibration parameters (`camCal`) and camera calibration errors/deviations (`errCal`) for each view. For further detail on the calibration routine, refer to the *OpenCV* documentation on camera calibration [[2]](#References).

In [5]:
# Retrieve camera calibration parameters
camCal, errCal = zip(*[
                       calibCamera(imgP, objP)
                       for (imgP, objP) in zip(imgPts, objPts)
                       ])

Let's extract the calibration parameters for each camera$-$the uncertainties are calculated as the standard deviations (multiplied by 3) for each parameter, which is collected in the `errCal` variable. Refer to `showCalibration()` (see: [calib_utilities.py](./calib_utilities.py)) for further detail.

In [6]:
# Calibration output parameters
interact(
         showCalibration, 
         ind=widgets.Dropdown(options=[1, 2, 3], description='Camera'),
         camCal=fixed(camCal),
         errCal=fixed(errCal)
         );

interactive(children=(Dropdown(description='Camera', options=(1, 2, 3), value=1), Output()), _dom_classes=('wi…

We can also visualize where the reprojections of the object points lie on the images as well. This is calculated by taking the initially known calibration plate world coordinates (`objPts`) and projecting them onto the image plane using the calculated extrinsics, intrinsics, and distortion coefficients. The projection process is done using `projectPoints()` from the *OpenCV* library. The mean pixel errors shown below refer to the mean error in how closely the reprojection points (blue crosses) match the initially found dots (red circles) for each view.

In [7]:
# Visualizing the results through an interactive plot
interact(
         showReprojection, 
         cam=widgets.Dropdown(options=[1, 2, 3], description='Camera'), 
         view=widgets.IntSlider(min=1, max=3, step=1, description='View'),
         imgPts=fixed(imgPts),
         objPts=fixed(objPts),
         camCal=fixed(camCal),
         errCal=fixed(errCal)
         );

interactive(children=(Dropdown(description='Camera', options=(1, 2, 3), value=1), IntSlider(value=1, descripti…

## Save dataset

Let's save the calibration parameters in the `/outputs` folder for further processing.

In [8]:
# Save results
[
 np.savez(f'{out_fld}/cam{i+1}_calibration.npz', cal=camCal[i], err=errCal[i])
 for i, _ in enumerate(camCal)
 ];

## Next steps
After initial camera calibration, the next step in the workflow is [pb02_preprocessing.ipynb](./pb02_preprocessing.ipynb).

## References

1. Zhang, Z. A flexible new technique for camera calibration. *IEEE Trans. Pattern Anal. Machine Intell.* 22, 1330–1334 (2000). doi: [10.1109/34.888718](https://doi.org/10.1109/34.888718).

2. *OpenCV*. Available at: https://docs.opencv.org/3.2.0/d9/d0c/group__calib3d.html.

---

**Author**: Naveed Rahman, Purdue University, 23 March 2022

---